# MOD16

In [ ]:
# Import relevant libraries
import csv
import ee

In [ ]:
# Load MOD16 ET dataset from Earth Engine
mod16 = ee.ImageCollection('MODIS/061/MOD16A2GF').filterBounds(region) # region defined in WaPOR section above (region = your point coords)

# Define the function to process MOD16 data
def process_mod16(image):

    # Exclude invalid values (water bodies, urban areas, etc.)
    mask = image.select('ET').lt(65529)  # Adjust this condition based on valid range
    et_mm = image.select('ET').updateMask(mask).multiply(0.1)  # Convert to mm after masking

    # Calculate monthly mean ET
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    mean_value = et_mm.reduceRegion(reducer=ee.Reducer.mean(), geometry=region, scale=500).getInfo()

    return {'date': date.getInfo(), 'MOD16': mean_value.get('ET')}


In [ ]:
# Apply the function to each MOD16 image in the collection
mod16_means = []
# Retrieve the list of images in the collection for your specified dates
mod16_images = mod16.filterDate('2014-06-01', '2023-09-01').toList(mod16.size())

# Loop through the list of images
for i in range(mod16_images.size().getInfo()):
    image = ee.Image(mod16_images.get(i))
    monthly_mean = process_mod16(image)
    mod16_means.append(monthly_mean)

# Write MOD16 data to CSV
with open('mod16_means.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['date', 'MOD16'])
    writer.writeheader()
    writer.writerows(mod16_means)

print("MOD16 data summarized and saved to mod16_means.csv")

In [ ]:
# Import relevant libraries
import csv
import ee
import pandas as pd

# Initialize the Earth Engine module
ee.Initialize()

In [ ]:
# Load MOD16 ET dataset from Earth Engine
mod16 = ee.ImageCollection('MODIS/061/MOD16A2GF').filterBounds(region)  # Using your predefined region above

# Define function to process MOD16 data
def process_mod16(image):
    # Exclude invalid values (masking out fill values)
    mask = image.select('ET').lt(65529)  # Adjust this based on valid ET values
    et_mm = image.select('ET').updateMask(mask).multiply(0.1)  # Convert to mm after masking

    # Calculate the month and year from the image date
    date = ee.Date(image.get('system:time_start'))
    date_str = date.format('YYYY-MM').getInfo()  # Get the date as Year-Month

    # Calculate the spatial mean ET for the image
    mean_value = et_mm.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region,  # Use your predefined region
        scale=500,  # Use a reasonable scale; adjust if needed
        maxPixels=1e6,  # Set a limit for maxPixels
        bestEffort=False  # Disable best effort to focus on the exact region
    ).getInfo()

    # Check if 'ET' key exists in mean_value and if it's not None
    # If 'ET' is None or not found, return 0 for MOD16
    et_value = mean_value.get('ET')
    return {'date': date_str, 'MOD16': et_value if et_value is not None else 0}

# Initialize a dictionary to accumulate monthly totals
monthly_totals = {}

# Get the list of images in the collection for the specified date range
image_list = mod16.filterDate('2014-06-01', '2023-09-01').toList(mod16.size())

# Iterate through the MOD16 images and accumulate monthly means
for i in range(image_list.size().getInfo()):
    image = ee.Image(image_list.get(i))  # Get the image from the list
    monthly_mean = process_mod16(image)
    date_key = monthly_mean['date']

    if date_key not in monthly_totals:
        monthly_totals[date_key] = 0
    # Add the spatial mean value (in mm) to the total for the month
    monthly_totals[date_key] += monthly_mean['MOD16']

In [ ]:
# Convert to a DataFrame for easy manipulation and export
mod16_monthly_df = pd.DataFrame(list(monthly_totals.items()), columns=['date', 'MOD16'])

# Write MOD16 monthly totals to CSV
mod16_monthly_df.to_csv('monthly_mod16_data.csv', index=False)